In [1]:
# Imports
import numpy as np
import uuid
from qsi.coordinator import Coordinator
from qsi.state import State, StateProp

In [2]:
# Create coordinator and start the modules:
coordinator = Coordinator(port=10000)
# Coherent source
cs  = coordinator.register_component(module="coherent_source.py", runtime="python")
fib = coordinator.register_component(module="fiber.py", runtime="python")
# Run the coordinator process
coordinator.run()

{'alpha': {'value': None, 'type': 'complex'}}
{'length': {'value': None, 'type': 'number'}, 'n': {'value': None, 'type': 'number'}}


In [3]:
# Set the alpha for the coherent source
cs.set_param("alpha", 1+0j)
cs.send_params()

# Set the length and refractive index for the fiber
fib.set_param("length", 10)
fib.set_param("n", 1.45)
fib.send_params()

In [4]:
# Generate the space for coherent state
coherent_state_prop = StateProp(
    state_type="light",
    truncation=10,
    wavelength=1550,
    polarization="R"
)
coherent_state = State(coherent_state_prop)

In [5]:
# Get the Kraus operators for the generation of the coherent state
response, operators = cs.channel_query(
    coherent_state, {"input": coherent_state_prop.uuid}
)
coherent_state.apply_kraus_operators(operators, [coherent_state_prop])
if response["error"]>0:
    print(f"Error due to truncation: {response["error"]}")

Error due to truncation: 1.1142547828857374e-07


In [6]:
# Get the Kraus operators for the fiber
response, operators = fib.channel_query(
    coherent_state, {"input": coherent_state_prop.uuid}
)
coherent_state.apply_kraus_operators(operators, [coherent_state_prop])